In [ ]:
from codegen import Codebase

from codegen.sdk.enums import ProgrammingLanguage

%load_ext autoreload
%autoreload 2

In [ ]:
codebase = Codebase(
    repo_path="./",
    programming_language=ProgrammingLanguage.TYPESCRIPT,
)

In [ ]:
function_calls = codebase.get_function("processUserData").function_calls
promise_chain = None

# Find the promise chain
for function_call in function_calls:
    if function_call.name == "then":
        promise_chain = function_call.get_promise_chain()
        break


print("\n=== Promise Chain Analysis ===")
print(f"Base Chain: {[call.name for call in promise_chain.base_chain]}")
print(f"Then Chain: {[call.name for call in promise_chain.then_chain]}")
print(f"Has Catch: {promise_chain.has_catch_call} -> {promise_chain.catch_call.name if promise_chain.catch_call else None}")
print(f"Has Finally: {promise_chain.has_finally_call} -> {promise_chain.finally_call.name if promise_chain.finally_call else None}")

## Test 1: Long Promise Chains

```typescript
// Example of nested .then() calls
async function getUserDataAndPosts(userId: number): Promise<void> {
    fetchUserData(userId)
        .then((user) => {
            console.log('User:', user);
            return fetchUserPosts(user.id);
        })
        .then((posts) => {
            console.log('Posts:', posts);
            return fetchPostComments(posts[0].id);
        })
        .then((comments) => {
            console.log('Comments:', comments);
        })
        .catch((error) => {
            console.error('Error:', error.message);
        });
}
```

In [ ]:
promise_chain.convert_to_async_await()
codebase.commit()

Converts to:

```typescript
function ensureTable(tableName, schemaName, trxOrKnex) {
    const lockTable = getLockTableName(tableName);
    return getSchemaBuilder(trxOrKnex, schemaName)
      .hasTable(tableName)
      .then((exists) => {
        return !exists && _createMigrationTable(tableName, schemaName, trxOrKnex);
      })
      .then(() => {
        return getSchemaBuilder(trxOrKnex, schemaName).hasTable(lockTable);
      })
      .then((exists) => {
        return (
          !exists && _createMigrationLockTable(lockTable, schemaName, trxOrKnex)
        );
      })
      .then(() => {
        return getTable(trxOrKnex, lockTable, schemaName).select('*');
      })
      .then((data) => {
        return (
          !data.length && _insertLockRowIfNeeded(tableName, schemaName, trxOrKnex)
        );
      });
  }
```

In [ ]:
codebase.reset()

In [ ]:
function_calls = codebase.get_function("ensureTable").function_calls
promise_chain = None

# Find the promise chain
for function_call in function_calls:
    if function_call.name == "then":
        promise_chain = function_call.get_promise_chain()
        break


print("\n=== Promise Chain Analysis ===")
print(f"Base Chain: {[call.name for call in promise_chain.base_chain]}")
print(f"Then Chain: {[call.name for call in promise_chain.then_chain]}")
print(f"Has Catch: {promise_chain.has_catch_call} -> {promise_chain.catch_call.name if promise_chain.catch_call else None}")
print(f"Has Finally: {promise_chain.has_finally_call} -> {promise_chain.finally_call.name if promise_chain.finally_call else None}")


# Test 2: Long Promise Chain from [knex](https://knexjs.org/)


```typescript
 function ensureTable(tableName, schemaName, trxOrKnex) {
    const lockTable = getLockTableName(tableName);
    return getSchemaBuilder(trxOrKnex, schemaName)
      .hasTable(tableName)
      .then((exists) => {
        return !exists && _createMigrationTable(tableName, schemaName, trxOrKnex);
      })
      .then(() => {
        return getSchemaBuilder(trxOrKnex, schemaName).hasTable(lockTable);
      })
      .then((exists) => {
        return (
          !exists && _createMigrationLockTable(lockTable, schemaName, trxOrKnex)
        );
      })
      .then(() => {
        return getTable(trxOrKnex, lockTable, schemaName).select('*');
      })
      .then((data) => {
        return (
          !data.length && _insertLockRowIfNeeded(tableName, schemaName, trxOrKnex)
        );
      });
  }
````

In [ ]:
promise_chain.convert_to_async_await()
codebase.commit()

Converts to:

```typescript

async function ensureTable(tableName, schemaName, trxOrKnex) {
    const lockTable = getLockTableName(tableName);
    let exists = await getSchemaBuilder(trxOrKnex, schemaName)
      .hasTable(tableName);
        await !exists && _createMigrationTable(tableName, schemaName, trxOrKnex);

        exists = await getSchemaBuilder(trxOrKnex, schemaName).hasTable(lockTable);

        await (
          !exists && _createMigrationLockTable(lockTable, schemaName, trxOrKnex)
        );

        let data = await getTable(trxOrKnex, lockTable, schemaName).select('*');

    return (
          !data.length && _insertLockRowIfNeeded(tableName, schemaName, trxOrKnex)
        );
  }
```
